<a href="https://colab.research.google.com/github/nicoleolivetto/CNN/blob/main/RNN_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("Women_clothing_reviews.csv").head(500)

In [7]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [8]:
print(f'Shape of data: {df.shape}')
# Find the number of missing values
print(df.info())

Shape of data: (500, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               500 non-null    int64 
 1   Clothing ID              500 non-null    int64 
 2   Age                      500 non-null    int64 
 3   Title                    403 non-null    object
 4   Review Text              481 non-null    object
 5   Rating                   500 non-null    int64 
 6   Recommended IND          500 non-null    int64 
 7   Positive Feedback Count  500 non-null    int64 
 8   Division Name            500 non-null    object
 9   Department Name          500 non-null    object
 10  Class Name               500 non-null    object
dtypes: int64(6), object(5)
memory usage: 43.1+ KB
None


In [9]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Review Text'].astype(str).str.lower())

total_words = len(tokenizer.word_index)+1
tokenized_sentences = tokenizer.texts_to_sequences(df['Review Text'].astype(str))

In [10]:
# Slash sequences into n gram sequence
input_sequences = list()
for i in tokenized_sentences:
    for t in range(1, len(i)):
        n_gram_sequence = i[:t+1]
        input_sequences.append(n_gram_sequence)

# Pre pad with max sequence length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [11]:
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 40, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(250)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=4, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=10, verbose=1, callbacks=[earlystop])

Epoch 1/10
919/919 [==============================] - 555s 598ms/step - loss: 6.0389 - accuracy: 0.0577
Epoch 2/10
919/919 [==============================] - 541s 589ms/step - loss: 5.4183 - accuracy: 0.1026
Epoch 3/10
919/919 [==============================] - 542s 589ms/step - loss: 4.9492 - accuracy: 0.1385
Epoch 4/10
919/919 [==============================] - 540s 588ms/step - loss: 4.5865 - accuracy: 0.1598
Epoch 5/10
919/919 [==============================] - 540s 588ms/step - loss: 4.2319 - accuracy: 0.1822
Epoch 6/10
919/919 [==============================] - 533s 580ms/step - loss: 3.8631 - accuracy: 0.2089
Epoch 7/10
919/919 [==============================] - 524s 570ms/step - loss: 3.4956 - accuracy: 0.2510
Epoch 8/10
495/919 [===============>..............] - ETA: 4:01 - loss: 3.0779 - accuracy: 0.3109